# ALGOBOWL - Heuristics for finding solutions
## Authors: James Mach, Hayden Sather, Harrison Magee
### Group 16 - Team O(G)
### Section B

This cell below helps with using Github

In [1]:
import numpy as np

# helps read in the files
class Subset:
    
    def __init__(self, subset, weight, ID, calculated_weight):
        self.subset = subset
        self.weight = weight
        self.ID = ID
        self.calculated_weight = calculated_weight
        
    def get_subset(self):
        return self.subset
    
    def get_weight(self):
        return self.weight
    
    def get_ID(self):
        return self.ID
    
    def contains(self, n):
        contains = False
        for m in subset:
            if m == n:
                contains = True
        return contains

    def get_size(self):
        return len(self.subset)
    
    def to_string(self):
        output = ""
        for n in self.subset:
            output = output+str(n)+" "
        output = output + "\n"+str(self.weight)
        return output
    
    def find_calculated_weight(self, remaining_numbers):
        number_matches = 0
        for num in self.subset:
            if num in remaining_numbers:
                number_matches += 1
        if number_matches == 0:
            self.calculated_weight = float("inf")
        else:
            self.calculated_weight = self.weight / number_matches
                

In [2]:
# this is the class with objects ALGOBowl
class ALGOBowl:
    
    def __init__(self):
        self.n=0
        self.m=0
        self.subsets = []  # all subsets in the problem
        self.solution = [] # = [[IDs of subsets], total_weight,[subsets in solution]]

    # reads input file and places into the subset    
    def read_input(self, filename):
        file = open(filename,'r')
        self.n = int(file.readline())
        self.m = int(file.readline())
        for i in range(1,self.m+1):
            temp = []
            for string in file.readline().split(" "):
                if(string == "\n"):
                    continue
                else:
                    temp.append(int(string))
            weight = int(file.readline())
            ss = Subset(temp, weight, i, 0)
            self.subsets.append(ss)
    
    # just a print function
    def display_data(self):
        print(self.n)
        print(self.m)
        for s in self.subsets:
            print(s.to_string())

    # not sure if usefull
    #     def get_subsets():
    #         sets = []
    #         for s in subsets:
    #             sets.append(s.get_subset())
    #         return sets
    
    # returns list of subsets that have unique values
    def find_unique_sets(self):
        counts = np.zeros(self.n)
        unique_nums = []
        unique_sets = []
        
        for s in self.subsets:
            for n in s.get_subset():
                counts[n-1]=counts[n-1]+1
        for i in range(0,self.n):
            if counts[i]==1:
                unique_nums.append(i+1)
        for s in self.subsets:
            for n in s.get_subset():
                for m in unique_nums:
                    if n == m:
                        unique_sets.append(s)
        return unique_sets
    
#debug/performance testing methods
    def sum_all_weights(self):
        sum_weights = 0
        for s in self.subsets:
            sum_weights = sum_weights + s.get_weight()
        return sum_weights
    
#find how many numbers are repersented more than once
    def count_overlaps(self):
        overlaps = 0
        counts = np.zeros(self.n)
        for s in self.subsets:
            for n in s.get_subset():
                counts[n-1]=counts[n-1]+1
        for n in counts:
            if n>1:
                overlaps = overlaps+1
        return overlaps
    
#prints solution, Debug = 1 shows all subsets, Debug = 2 shows all subsets plus other stats   
    def print_solution(self, Debug = None):
        if Debug == None:
            output = str(self.solution[1])+"\n"
            for s in self.solution[0]:
                output = output + str(s) + " "
            return output
        elif Debug == 1:
            output = str(self.solution[1])+"\n"
            for s in self.solution[0]:
                output = output + str(s) + " "
            for s in self.solution[2]:
                output = output + "\n" +str(s)
            return output
        elif Debug == 2:
            output = str(self.solution[1])+"\n"
            for s in self.solution[0]:
                output = output + str(s) + " "
            output = output + "\n\n\n" + "Number of overlaps : " + str(self.count_overlaps())
            ratio = self.solution[1]/self.sum_all_weights()
            output = output + "\n" + "Solution weight/All weights : " + str(ratio)
            return output

# writes the sets we get with the weights to an output
    def write_output(self, filename):
        file = open(filename, "w")
        file.write(self.print_solution())
        file.flush()
        file.close()
        
    #CHEAPEST-FIRST ALGORITHM
    def run_cheapest_first(self):
        IDs = []
        solution = []
        total_weight=0
        nums = set()
        self.subsets.sort(key = lambda x : x.get_weight())
        i =0
        while len(nums)<self.n:
            IDs.append(self.subsets[i].get_ID())
            solution.append(self.subsets[i].get_subset())
            total_weight = total_weight + self.subsets[i].get_weight()
            nums.update(self.subsets[i].get_subset())
            i = i + 1
        IDs.sort()
        self.solution = [IDs, total_weight, solution]
        return [IDs, total_weight, solution]
         
    
    # MOST DISJOINT ALGORITHM
    def disjoint(self, *args, sets = None):
        if sets == None:
            nums = set()
            for s in args:
                for n in s:
                    if n in nums:
                        return False
                    else:
                        nums.add(n)
            return True
        else:
            nums = set()
            for s in sets:
                for n in s:
                    if n in nums:
                        return False
                    else:
                        nums.add(n)
            for s in args:
                for n in s:
                    if n in nums:
                        return False
                    else:
                        nums.add(n)
            return True
   
    def run_most_disjoint(self):
        del self.solution[:] # clear solution
        IDs = []
        solution = []
        total_weight=0
        nums = set()
        for s in self.subsets:
            if self.disjoint(s.get_subset(), sets = solution):
                solution.append(s.get_subset())
                IDs.append(s.get_ID())
                total_weight = total_weight + s.get_weight()
                nums.update(s.get_subset())
        if len(nums) != self.n:
            rest = self.find_needed_subsets(nums)
            for ids in rest[0]:
                IDs.append(ids)
            total_weight = total_weight + rest[1]
            for s in rest[2]:
                solution.append(s)
        IDs.sort()
        self.solution = [IDs, total_weight, solution]
        return [IDs, total_weight, solution]


    def find_needed_subsets(self, nums):#helper function for disjoint algorithm^^^^
        IDs = []
        solution = []
        total_weight =0
        needed_numbers = set()
        for n in range(1,self.n+1):
            if not n in nums:
                needed_numbers.add(n)
        for s in self.subsets:
            for n in s.get_subset():
                if n in nums:
                    continue
                if n in needed_numbers:
                    IDs.append(s.get_ID())
                    solution.append(s.get_subset())
                    total_weight = total_weight + s.get_weight()
                    nums.add(n)
                    break
        return [IDs, total_weight, solution]
    
    # LEAST WEIGHT PER ITEM ALGORITM
    def run_cheapest_per_item_first(self):
        IDs = []
        solution = []
        total_weight=0
        nums = set()
        self.subsets.sort(key = lambda x : x.get_weight()/len(x.get_subset()))
        i =0
        while len(nums)<self.n:
            IDs.append(self.subsets[i].get_ID())
            solution.append(self.subsets[i].get_subset())
            total_weight = total_weight + self.subsets[i].get_weight()
            nums.update(self.subsets[i].get_subset())
            i = i + 1
        IDs.sort()
        self.solution = [IDs, total_weight, solution]
        return [IDs, total_weight, solution]
    
    
    # CALCULATED WEIGHT ALGORITHM
    def remove_element_from_list(self, list_in, element):
        list_in[:] = [x for x in list_in if x != element]
        return list_in

    def remove_elements_of_subset_from_list(self, subset, list_to_take_from):
        for number in subset:
            list_to_take_from = self.remove_element_from_list(list_to_take_from, number)
        return list_to_take_from

    def update_all_calculated_weights(self, numbers_left):
        for subset in self.subsets:
            subset.find_calculated_weight(numbers_left)
            
    def run_calculated_weight(self):
        print("starting")
        IDs = []
        solution = []
        total_weight = 0
        numbers_to_cover = list(range(1, self.n))
        
        while(len(numbers_to_cover) > 0):
            self.update_all_calculated_weights(numbers_to_cover)
            self.subsets.sort(key = lambda x : x.calculated_weight)
            current_subset = self.subsets[0]
            IDs.append(current_subset.get_ID())
            solution.append(current_subset.get_subset())
            total_weight = total_weight + current_subset.get_weight()
            numbers_to_cover = self.remove_elements_of_subset_from_list(current_subset.subset, numbers_to_cover)
            del self.subsets[0]
        
        IDs.sort()
        self.solution = [IDs, total_weight, solution]
        return [IDs, total_weight, solution]
            
            

In [3]:
#main
index = 4
inputs = ["easy", "input_10", "input_50", "input_100", "input_500","disjoint"]
ext = ".txt"
output = "_output.txt"
test = ALGOBowl()
test.read_input(inputs[index]+ext)
# test.display_data()
# test.run_most_disjoint()
test.run_calculated_weight()
print(test.print_solution(Debug = 2))
test.write_output(inputs[index]+output)

starting
167
17 50 70 92 126 148 168 199 208 210 231 


Number of overlaps : 500
Solution weight/All weights : 0.002677741076868807


In [4]:
# Test the tough input

test = ALGOBowl()
test.read_input("tough_input.txt")
test.run_calculated_weight()
print(test.print_solution(Debug = 2))
test.write_output("tough_input_output.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'tough_input.txt'

## Generating the Outputs from the other Groups inputs

In [ ]:
inputs = "input_group"
ext = ".txt"
output = "_output.txt"
for i in range (113, 133):
    test = ALGOBowl()
    test.read_input(inputs + i + ext)
    test.run_calculated_weight()

print(test.print_solution(Debug = 2))
test.write_output(inputs[index]+output)